Groupby ile sütuna göre gruplandırma yapılır. Birden çok sütuna göre gruplama mümkün olsa da bu iş için **pivot tables** kullanılır. Bu iş için örnek olarak titanik yolcularının verisini kullanacağız.

In [1]:
import pandas as pd

titanic = pd.read_csv("../Data/Titanic.csv")
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Öncelikle groupby kullanarak Titanik'te hayatta kalanların cinsiyet oranlarına bakalım.

In [2]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


Parametreye `sex` sütununu vererek sex'e göre gruplama yapacağımızı belirttik ve `survived` verisinin `mean`ını (ortalamasını) aldık.

Şimdi gruplandırma işlemini iki sütunla yapalım. Mesela biletin kaçıncı sınıf olduğunun da ölüme ne kadar etkisi olduğunu öğrenelim.

In [3]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


## Pivot Table Kullanımı

Gruplandırmayı çok sütunla yaptığımızda groupby biraz uğraştırdı. Bu amaç için pivot table kullanılır. Önceki yaptığımız işlemin aynısını pivot table ile aşağıdaki şekilde yapabiliriz.

In [4]:
titanic.pivot_table('survived', index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


`pivot_table` ilk parametre olarak aggregation'un uygulanacağı sütunu alır. Sonra `index` parametresinde index olarak alınacak, `columns` parametresinde de sütun olarak alınacak sütunları alır.

### Multilevel Pivot Tables

GroupBy'da olduğu gibi pivot table'larda da MultiIndex'li veri oluşturabiliriz. Aşağıdaki örnekte `pd.cut` kullanarak yaş sütununu yaşlara bölüyoruz. Sonra bu veriyi index olarak kullanılması için ikinci sütuna veriyoruz.

In [5]:
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

### Pivot Table Parametreleri

1. fill_value:

`fill_value`, NaN olan değerlerin yerine ne konarak işleme devam edileceğini belirtir.

2. dropna

True verilmesi, NaN değerlerin bulunduğu satırların yok sayılarak işlem yapılmasını sağlar.

3. aggfunc

`aggfunc` ile hangi sütun üzerinde hangi aggregation işlemleri yapacağımızı belirtebiliriz.

In [6]:
titanic.pivot_table(index='sex', columns='class', aggfunc={'survived':'mean', 'fare':sum})

fare                        survived                    
class       First     Second      Third     First    Second     Third
sex                                                                  
female  9975.8250  1669.7292  2321.1086  0.968085  0.921053  0.500000
male    8201.5875  2132.1125  4393.5865  0.368852  0.157407  0.135447

4. margins

True verildiğinde sütun ve satırın sonuna işlemin toplu sonucunu verir (bölümlenmemiş halini).

In [7]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838
